# Deep Neural Networks with Regularization


In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

Load the data

In [2]:
pickle_file = ".\gmql_data\kirc_data_divided_deneme.p"

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)
  print('Training set', valid_dataset.shape, valid_labels.shape)


Training set (363, 59) (363,)
Test set (122, 59) (122,)
Training set (121, 59) (121,)


Reformat into a shape that's more adapted to the models we're going to train:
data as a flat matrix,
labels as float 1-hot encodings.

In [3]:
features = 59
num_labels = 2

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, features)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)
print('Training set', valid_dataset.shape, valid_labels.shape)

Training set (363, 59) (363, 2)
Test set (122, 59) (122, 2)
Training set (121, 59) (121, 2)


In [4]:
batch_size = 50
reg = 3e-3

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are attached to the graph.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, features))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([features, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
  
    # Training computation.
    # Inputs multiplied with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy. We take the average of this cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits,labels =tf_train_labels))
    #Regularization is added to loss
    loss = loss + reg * tf.nn.l2_loss(weights)
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    




In [5]:
num_steps = 4000

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the biases. 
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
  
        if (step % 100 == 0):
            print('Mini batch loss at step %d: %f' % (step, l))
            print('Minibatch training accuracy: %.1f%%' % accuracy(predictions, train_labels[:batch_size, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Mini batch loss at step 0: 0.909946
Minibatch training accuracy: 36.0%
Validation accuracy: 64.5%
Mini batch loss at step 100: 0.148674
Minibatch training accuracy: 90.0%
Validation accuracy: 89.3%
Mini batch loss at step 200: 0.150757
Minibatch training accuracy: 90.0%
Validation accuracy: 89.3%
Mini batch loss at step 300: 0.171790
Minibatch training accuracy: 88.0%
Validation accuracy: 89.3%
Mini batch loss at step 400: 0.207899
Minibatch training accuracy: 88.0%
Validation accuracy: 90.1%
Mini batch loss at step 500: 0.248764
Minibatch training accuracy: 88.0%
Validation accuracy: 90.1%
Mini batch loss at step 600: 0.289417
Minibatch training accuracy: 88.0%
Validation accuracy: 90.1%
Mini batch loss at step 700: 0.332127
Minibatch training accuracy: 88.0%
Validation accuracy: 90.1%
Mini batch loss at step 800: 0.370728
Minibatch training accuracy: 86.0%
Validation accuracy: 90.1%
Mini batch loss 